In [1]:
import os
import glob
import numpy as np
import pyvips
from tqdm import tqdm
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [2]:
!rm /BrainSeg/ground_truth_masks/.ipynb_checkpoints -rf

In [3]:
TRUTH_DIR = '/BrainSeg/ground_truth_masks/'
TEST_DIR = '/BrainSeg/Classify_Results/ResNet/GMBMasks/'
SAVE_FILENAME = 'AccuracyResults_pixel.csv' # Under TEST_DIR
SAVE_FILENAME_LATEX = 'AccuracyResults_pixel.txt' # Under TEST_DIR

In [4]:
truth_images = sorted(os.listdir(TRUTH_DIR))
IMAGEDIMNORM = np.zeros((30,2), np.int)
for i in range(0, len(truth_images), 3):
    img = pyvips.Image.new_from_file(TRUTH_DIR + truth_images[i])
    width = img.width
    height = img.height
    IMAGEDIMNORM[i//3,0] = width
    IMAGEDIMNORM[i//3,1] = height
# print(IMAGEDIMNORM)
print(IMAGEDIMNORM.shape)

(30, 2)


In [5]:
image_names = glob.glob(TRUTH_DIR + "*Grey.png")
image_names = [imagename.split('/')[-1] for imagename in image_names]
image_names = [imagename.split('-Grey')[0] for imagename in image_names]
image_names = sorted(image_names)
print(len(image_names))

30


In [6]:
results = np.zeros((len(image_names), 8), dtype='int')

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
    # Filename for testing and ground truth images
    test_grey_img_name = imagename + "-Grey.png"
    test_white_img_name = imagename + "-White.png"
    truth_grey_img_name = imagename + "-Grey.png"
    truth_white_img_name = imagename + "-White.png"

    ##### Testing Grey Matter Mask #####
    # Load images
    TEST_img = np.array(Image.open(TEST_DIR + test_grey_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_grey_img_name)
#     (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
#     truth_img = truth_img.resize((width, height), Image.BILINEAR)
    truth_img = np.array(truth_img)
    truth_width = truth_img.shape[0]
    truth_height = truth_img.shape[1]
    
    test_img = np.zeros([truth_width,truth_height], bool)
    for w in range(0,truth_width-128,128):
        for h in range(0,truth_height-128,128):
            test_img[w:w+128, h:h+128] = TEST_img[w//128,h//128]
    
    # Check dimensions
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1] and test_img.shape[1] == IMAGEDIMNORM[i, 0]):
        print('\t' + imagename + ' grey dimension WRONG!!!')

    # Update tqdm descriptions
    t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
      imagename, str(test_img.shape), str(truth_img.shape)) )
    t.refresh()
    t.write("", end=' ')

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    grey_TP = np.multiply(test_img, truth_img).sum()
    grey_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    grey_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    grey_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    grey_total = width*height
    if not (grey_total == grey_TP+grey_FP+grey_FN+grey_TN):
        print('\t' + imagename + ' grey matter results WRONG!!!')

    ##### Testing White Matter Mask #####
    # Load images
    TEST_img = np.array(Image.open(TEST_DIR + test_white_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_white_img_name)
    truth_img = np.array(truth_img)
    
    truth_img = np.array(truth_img)
    truth_width = truth_img.shape[0]
    truth_height = truth_img.shape[1]
    
    test_img = np.zeros([truth_width,truth_height], bool)
    for w in range(0,truth_width-128,128):
        for h in range(0,truth_height-128,128):
            test_img[w:w+128, h:h+128] = TEST_img[w//128,h//128]

    # Check dimensions
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1] and test_img.shape[1] == IMAGEDIMNORM[i, 0]):
        print('\t' + imagename + ' white dimension WRONG!!!')

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    white_TP = np.multiply(test_img, truth_img).sum()
    white_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    white_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    white_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    white_total = width*height
    if not (white_total == white_TN+white_FP+white_FN+white_TP):
        print('\t' + imagename + ' white matter results WRONG!!!')

    # Save results: grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
    results[i, :] = [grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP]

    t.update()
t.close()

with open(TEST_DIR + SAVE_FILENAME, "w") as f:
    f.write('Image Name,Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
          'White_TN (px),White_FP (px),White_FN (px),White_TP (px),' + 
          'G_Accuracy (%),G_Misclassification Rate (%),G_Sensitivity (%),' +
          'G_Specificity (%),G_Precision (%),G_Prevalence (%),' + 
          'W_Accuracy (%),W_Misclassification Rate (%),W_Sensitivity (%),' +
          'W_Specificity (%),W_Precision (%),W_Prevalence (%)\n') 
    accu_result = np.zeros((len(image_names), 12))
    for i, imagename in enumerate(image_names):
        total = results[i, 0]+results[i, 1]+results[i, 2]+results[i, 3]
        accu_result[i, 0]  = (results[i, 0]+results[i, 3])/total*100
        accu_result[i, 1]  = (results[i, 1]+results[i, 2])/total*100
        accu_result[i, 2]  = (results[i, 3]/(results[i, 2]+results[i, 3]))*100
        accu_result[i, 3]  = (results[i, 0]/(results[i, 0]+results[i, 1]))*100
        accu_result[i, 4]  = (results[i, 3]/(results[i, 1]+results[i, 3]))*100
        accu_result[i, 5]  = (results[i, 2]+results[i, 3])/total*100
        total = results[i, 4]+results[i, 5]+results[i, 6]+results[i, 7]
        accu_result[i, 6]  = (results[i, 4]+results[i, 7])/total*100
        accu_result[i, 7]  = (results[i, 5]+results[i, 6])/total*100
        accu_result[i, 8]  = (results[i, 7]/(results[i, 6]+results[i, 7]))*100
        accu_result[i, 9]  = (results[i, 4]/(results[i, 4]+results[i, 5]))*100
        accu_result[i, 10] = (results[i, 7]/(results[i, 5]+results[i, 7]))*100
        accu_result[i, 11] = (results[i, 6]+results[i, 7])/total*100
        f.write('%s,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
            (imagename, results[i, 0], results[i, 1], results[i, 2], results[i, 3], 
                results[i, 4], results[i, 5], results[i, 6], results[i, 7], 
                accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], 
                accu_result[i, 3], accu_result[i, 4], accu_result[i, 5], 
                accu_result[i, 6], accu_result[i, 7], accu_result[i, 8], 
                accu_result[i, 9], accu_result[i, 10], accu_result[i, 11]))
    f.write('-' * 230 + '\n')
    f.write('Average,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
          (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), np.mean(results[:, 3]), 
              np.mean(results[:, 4]), np.mean(results[:, 5]), np.mean(results[:, 6]), np.mean(results[:, 7]), 
              np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), np.mean(accu_result[:, 2]), 
              np.mean(accu_result[:, 3]), np.mean(accu_result[:, 4]), np.mean(accu_result[:, 5]), 
              np.mean(accu_result[:, 6]), np.mean(accu_result[:, 7]), np.mean(accu_result[:, 8]), 
              np.mean(accu_result[:, 9]), np.mean(accu_result[:, 10]), np.mean(accu_result[:, 11])))


Image NA3777-02_AB, Test ((47177, 53784)), Ground Truth ((47177, 53784)):   0%|          | 0/30 [00:08<?, ?it/s]

Image NA4077-02_AB, Test ((46200, 73704)), Ground Truth ((46200, 73704)):   3%|▎         | 1/30 [05:20<2:18:11, 285.92s/it]

Image NA4092-02_AB, Test ((45178, 65736)), Ground Truth ((45178, 65736)):   7%|▋         | 2/30 [10:46<2:22:34, 305.53s/it]

Image NA4107-02_AB, Test ((36459, 67728)), Ground Truth ((36459, 67728)):  10%|█         | 3/30 [15:36<2:15:36, 301.34s/it]

Image NA4160-02_AB, Test ((41787, 59760)), Ground Truth ((41787, 59760)):  13%|█▎        | 4/30 [19:18<2:00:18, 277.62s/it]

...

Image NA4993-02_AB17-24, Test ((41543, 39840)), Ground Truth ((41543, 39840)):  87%|████████▋ | 26/30 [48:36<03:13, 48.49s/it]

Image NA5010-02_AB17-24, Test ((77386, 75695)), Ground Truth ((77386, 75695)):  90%|█████████ | 27/30 [49:25<02:12, 44.22s/it]

Image NA5015-02_AB17-24, Test ((40419, 55776)), Ground Truth ((40419, 55776)):  93%|█████████▎| 28/30 [58:34<06:38, 199.43s/it]

Image NA5029-02_AB17-24, Test ((43598, 53784)), Ground Truth ((43598, 53784)):  97%|█████████▋| 29/30 [59:23<02:34, 154.25s/it]

Image NA5029-02_AB17-24, Test ((43598, 53784)), Ground Truth ((43598, 53784)): 100%|██████████| 30/30 [1:00:06<00:00, 120.22s/it]


## IoU

In [7]:
#@title
SAVE_FILENAME = 'IoUResults.csv' # Under TEST_DIR
SAVE_FILENAME_LATEX = 'IoUResults.txt' # Under TEST_DIR

# Stores results: back_TN, back_FP, back_FN, back_TP, grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
conf_mat = np.zeros((len(image_names), 12), dtype='int')
# Stores accuracy results: back_IoU, back_F1, grey_IoU, grey_F1, white_IoU, white_F1
results = np.zeros((len(image_names), 6))
# Stores accuracy results: back_IoU_conf, back_F1_conf, grey_IoU_conf, grey_F1_conf, white_IoU_conf, white_F1_conf
results_conf = np.zeros((len(image_names), 6))

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
    # Filename for testing and ground truth images
    test_grey_img_name = imagename + "-Grey.png"
    test_white_img_name = imagename + "-White.png"
    test_back_img_name = imagename + "-Background.png"
    truth_grey_img_name = imagename + "-Grey.png"
    truth_white_img_name = imagename + "-White.png"
    truth_back_img_name = imagename + "-Background.png"

    ##### Testing Background Mask #####
    # Load images
    TEST_img = np.array(Image.open(TEST_DIR + test_back_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_back_img_name)
    (width, height) = (IMAGEDIMNORM[i, 0], IMAGEDIMNORM[i, 1])
    truth_img = np.array(truth_img)
    truth_width = truth_img.shape[0]
    truth_height = truth_img.shape[1]
    
    test_img = np.zeros([truth_width,truth_height], bool)
    for w in range(0,truth_width-128,128):
        for h in range(0,truth_height-128,128):
            test_img[w:w+128, h:h+128] = TEST_img[w//128,h//128]
    
    # Check dimensions and dtype
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1] and test_img.shape[1] == IMAGEDIMNORM[i, 0]):
        print('\t' + imagename + ' background dimension WRONG!!!')
    assert(truth_img.dtype == np.bool)

  # Update tqdm descriptions
    t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
        imagename, str(test_img.shape), str(truth_img.shape)) )
    t.refresh()
    t.write("", end=' ')

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    back_TP = np.multiply(test_img, truth_img).sum()
    back_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    back_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    back_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    back_total = width*height
    if not (back_total == back_TP+back_FP+back_FN+back_TN):
        print('\t' + imagename + ' background results WRONG!!!')

    # Calculate IoU and F1 score
    back_IoU = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
    back_F1 = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

    # Calculate IoU and F1 score using TP FP FN
    back_IoU_conf = back_TP / (back_TP+back_FP+back_FN)
    back_F1_conf = 2*back_TP / (2*back_TP+back_FP+back_FN)

    ##### Testing Grey Matter Mask #####
    # Load images
    TEST_img = np.array(Image.open(TEST_DIR + test_grey_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_grey_img_name)
    (width, height) = (IMAGEDIMNORM[i, 0], IMAGEDIMNORM[i, 1])
    truth_img = np.array(truth_img)
    
    truth_width = truth_img.shape[0]
    truth_height = truth_img.shape[1]
    
    test_img = np.zeros([truth_width,truth_height], bool)
    for w in range(0,truth_width-128,128):
        for h in range(0,truth_height-128,128):
            test_img[w:w+128, h:h+128] = TEST_img[w//128,h//128]

    # Check dimensions and dtype
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1] and test_img.shape[1] == IMAGEDIMNORM[i, 0]):
        print('\t' + imagename + ' grey dimension WRONG!!!')
    assert(truth_img.dtype == np.bool)

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    grey_TP = np.multiply(test_img, truth_img).sum()
    grey_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    grey_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    grey_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    grey_total = width*height
    if not (grey_total == grey_TP+grey_FP+grey_FN+grey_TN):
        print('\t' + imagename + ' grey matter results WRONG!!!')

    # Calculate IoU and F1 score
    grey_IoU = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
    grey_F1 = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

    # Calculate IoU and F1 score using TP FP FN
    grey_IoU_conf = grey_TP / (grey_TP+grey_FP+grey_FN)
    grey_F1_conf = 2*grey_TP / (2*grey_TP+grey_FP+grey_FN)

    ##### Testing White Matter Mask #####
    # Load images
    TEST_img = np.array(Image.open(TEST_DIR + test_white_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_white_img_name)
    (width, height) = (IMAGEDIMNORM[i, 0], IMAGEDIMNORM[i, 1])
    truth_img = np.array(truth_img)

    truth_width = truth_img.shape[0]
    truth_height = truth_img.shape[1]
    
    test_img = np.zeros([truth_width,truth_height], bool)
    for w in range(0,truth_width-128,128):
        for h in range(0,truth_height-128,128):
            test_img[w:w+128, h:h+128] = TEST_img[w//128,h//128]
            
    # Check dimensions and dtype
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1] and test_img.shape[1] == IMAGEDIMNORM[i, 0]):
        print('\t' + imagename + ' white dimension WRONG!!!')
    assert(truth_img.dtype == np.bool)

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    white_TP = np.multiply(test_img, truth_img).sum()
    white_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    white_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    white_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    white_total = width*height
    if not (white_total == white_TN+white_FP+white_FN+white_TP):
        print('\t' + imagename + ' white matter results WRONG!!!')

    # Calculate IoU and F1 score
    white_IoU = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
    white_F1 = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

    # Calculate IoU and F1 score using TP FP FN
    white_IoU_conf = white_TP / (white_TP+white_FP+white_FN)
    white_F1_conf = 2*white_TP / (2*white_TP+white_FP+white_FN)

    # Save results: back_TN, back_FP, back_FN, back_TP, grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
    conf_mat[i, :] = [back_TN, back_FP, back_FN, back_TP, grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP]
    # Stores accuracy results: back_IoU, back_F1, grey_IoU, grey_F1, white_IoU, white_F1
    results[i, :] = [back_IoU, back_F1, grey_IoU, grey_F1, white_IoU, white_F1]
    # Stores accuracy results: back_IoU_conf, back_F1_conf, grey_IoU_conf, grey_F1_conf, white_IoU_conf, white_F1_conf
    results_conf[i, :] = [back_IoU_conf, back_F1_conf, grey_IoU_conf, grey_F1_conf, white_IoU_conf, white_F1_conf]

    t.update()
t.close()

################################################################################
# Multiply results and results_conf by 100 (%)
results *= 100
results_conf *= 100

# Average results: Avg_IoU (%),Avg_IoU_conf (%),Avg_F1 (%),Avg_F1_conf (%)
avg_result = np.zeros((len(image_names), 4))
avg_result[:, 0] = np.mean(results[:, 0:6:2], axis=1)
avg_result[:, 1] = np.mean(results_conf[:, 0:6:2], axis=1)
avg_result[:, 2] = np.mean(results[:, 1:6:2], axis=1)
avg_result[:, 3] = np.mean(results_conf[:, 1:6:2], axis=1)

# Generate results file
with open(TEST_DIR + SAVE_FILENAME, "w") as f:
    f.write('Image Name,Back_TN (px),Back_FP (px),Back_FN (px),Back_TP (px),' +
          'Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
          'White_TN (px),White_FP (px),White_FN (px),White_TP (px),' + 
          'Back_IoU (%),Back_IoU_conf (%),Back_F1 (%),Back_F1_conf (%),' +
          'Grey_IoU (%),Grey_IoU_conf (%),Grey_F1 (%),Grey_F1_conf (%),' +
          'White_IoU (%),White_IoU_conf (%),White_F1 (%),White_F1_conf (%),' +
          'Avg_IoU (%),Avg_IoU_conf (%),Avg_F1 (%),Avg_F1_conf (%)\n') 

    for i, imagename in enumerate(image_names):
        f.write('%s,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
            (imagename, conf_mat[i, 0], conf_mat[i, 1], conf_mat[i, 2], conf_mat[i, 3], 
                conf_mat[i, 4], conf_mat[i, 5], conf_mat[i, 6], conf_mat[i, 7], 
                conf_mat[i, 8], conf_mat[i, 9], conf_mat[i, 10], conf_mat[i, 11], 
                results[i, 0], results_conf[i, 0], results[i, 1], results_conf[i, 1], 
                results[i, 2], results_conf[i, 2], results[i, 3], results_conf[i, 3], 
                results[i, 4], results_conf[i, 4], results[i, 5], results_conf[i, 5],
                avg_result[i, 0], avg_result[i, 1], avg_result[i, 2], avg_result[i, 3])) 
    f.write('-' * 230 + '\n')
    f.write('Average,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
          (np.mean(conf_mat[:, 0]), np.mean(conf_mat[:, 1]), np.mean(conf_mat[:, 2]), np.mean(conf_mat[:, 3]), 
              np.mean(conf_mat[:, 4]), np.mean(conf_mat[:, 5]), np.mean(conf_mat[:, 6]), np.mean(conf_mat[:, 7]), 
              np.mean(conf_mat[:, 8]), np.mean(conf_mat[:, 9]), np.mean(conf_mat[:, 10]), np.mean(conf_mat[:, 11]), 
              np.mean(results[:, 0]), np.mean(results_conf[:, 0]), np.mean(results[:, 1]), np.mean(results_conf[:, 1]), 
              np.mean(results[:, 2]), np.mean(results_conf[:, 2]), np.mean(results[:, 3]), np.mean(results_conf[:, 3]), 
              np.mean(results[:, 4]), np.mean(results_conf[:, 4]), np.mean(results[:, 5]), np.mean(results_conf[:, 5]),
              np.mean(avg_result[:, 0]), np.mean(avg_result[:, 1]), np.mean(avg_result[:, 2]), np.mean(avg_result[:, 3])))

# Generate results file for LaTeX
# with open(TEST_DIR + SAVE_FILENAME_LATEX, "w") as f:
#     f.write('Image Name,Back_IoU (%),Back_F1 (%),Grey_IoU (%),Grey_F1 (%),White_IoU (%),White_F1 (%),Avg_IoU (%),Avg_F1 (%)\n')
#     for i, imagename in enumerate(image_names):
#         f.write('%s & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\\hline\n' % 
#             (imagename.split('-')[0], results[i, 0], results[i, 1], results[i, 2], 
#                 results[i, 3], results[i, 4], results[i, 5], 
#                 avg_result[i, 0], avg_result[i, 2]))
#     f.write('Average & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\n' % 
#             (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), 
#              np.mean(results[:, 3]), np.mean(results[:, 4]), np.mean(results[:, 5]),
#              np.mean(avg_result[:, 0]), np.mean(avg_result[:, 2])))
print('Done!')

Image NA3777-02_AB, Test ((47177, 53784)), Ground Truth ((47177, 53784)):   0%|          | 0/30 [00:07<?, ?it/s]

Image NA4077-02_AB, Test ((46200, 73704)), Ground Truth ((46200, 73704)):   3%|▎         | 1/30 [02:14<1:00:00, 124.15s/it]

Image NA4092-02_AB, Test ((45178, 65736)), Ground Truth ((45178, 65736)):   7%|▋         | 2/30 [05:30<1:08:11, 146.11s/it]

Image NA4107-02_AB, Test ((36459, 67728)), Ground Truth ((36459, 67728)):  10%|█         | 3/30 [07:56<1:05:54, 146.48s/it]

Image NA4160-02_AB, Test ((41787, 59760)), Ground Truth ((41787, 59760)):  13%|█▎        | 4/30 [09:57<1:00:14, 139.00s/it]

...

Image NA5010-02_AB17-24, Test ((77386, 75695)), Ground Truth ((77386, 75695)):  90%|█████████ | 27/30 [53:45<04:22, 87.48s/it]

Image NA5015-02_AB17-24, Test ((40419, 55776)), Ground Truth ((40419, 55776)):  93%|█████████▎| 28/30 [1:06:54<10:02, 301.25s/it]

Image NA5029-02_AB17-24, Test ((43598, 53784)), Ground Truth ((43598, 53784)):  97%|█████████▋| 29/30 [1:08:48<04:04, 244.99s/it]

Image NA5029-02_AB17-24, Test ((43598, 53784)), Ground Truth ((43598, 53784)): 100%|██████████| 30/30 [1:10:39<00:00, 141.32s/it]

Done!


In [54]:
import pandas as pd
csv = pd.read_csv(TEST_DIR + SAVE_FILENAME)

In [55]:
csv

,Image Name,Back_TN (px),Back_FP (px),Back_FN (px),Back_TP (px),Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),White_TN (px),...,Grey_F1 (%),Grey_F1_conf (%),White_IoU (%),White_IoU_conf (%),White_F1 (%),White_F1_conf (%),Avg_IoU (%),Avg_IoU_conf (%),Avg_F1 (%),Avg_F1_conf (%)
0,NA3777-02_AB,1.391187e+09,17096437.0,62952878.0,1.066132e+09,1.495174e+09,68761632.0,38084246.0,935348192.0,2.078641e+09,...,94.6,94.6,91.8,91.8,95.72,95.72,91.52,91.52,95.57,95.57
1,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Average,1.391187e+09,17096437.0,62952878.0,1.066132e+09,1.495174e+09,68761632.0,38084246.0,935348192.0,2.078641e+09,...,94.6,94.6,91.8,91.8,95.72,95.72,91.52,91.52,95.57,95.57
